# SQLite database

In [20]:
import sqlite3
import pandas as pd

Create table Wheat Spotprice

In [6]:
con = sqlite3.connect('price_data.db')
cur = con.cursor()
cur.execute('''
CREATE TABLE If NOT EXISTS wheat_spotprice
    (date DATE unique,
    price real
    )
''')
con.commit()
con.close()

Populate table Wheat Spotprice

Check result:

In [25]:
def get_wheat_df():
    '''returns all entries from wheat_spotprice_table as dataframe'''
    con = sqlite3.connect('price_data.db')
    wheat_df = pd.read_sql('''
    Select * FROM wheat_spotprice''', con,  parse_dates=['date'])
    return wheat_df


In [26]:
wheat_df = check_wheat_spotprice()
wheat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213 entries, 0 to 212
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    213 non-null    object 
 1   price   213 non-null    float64
dtypes: float64(1), object(1)
memory usage: 3.5+ KB
